# INNIO Fleet Analysis

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from pprint import (pprint as pp, pformat as pf)
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import HTML, display
import ipywidgets as widgets

import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(3600)

### Filter Engines from installed fleet 

In [ ]:
def sfun(x):
    return all([
            ("Forsa Hartmoor" in str(x['IB Site Name'])), 
            #("Landshut" in str(x['IB Site Name'])), 
            (x['OperationalCondition'] != 'Decommissioned')
        ])

In [ ]:
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

In [ ]:
ddl = pd.DataFrame(fleet['serialNumber'] + ' - ' + fleet['IB Site Name'] + ' ' + fleet['Engine ID'])[0].to_list()
ddl = [(m, i) for i, m in enumerate(ddl)]
dl=widgets.Dropdown(options = ddl,value=0,description='Engine:',layout={'width':'max-content'},disabled=False)
display(dl)

In [ ]:
e=dmyplant2.Engine.from_fleet(mp,motor:=fleet.iloc[dl.value])
pd.DataFrame.from_dict(e.dash, orient='index').T

In [ ]:
from dfsm import msgFSM
#fsm = msgFSM(e, skip_day=7) #funktioniert nicht
#fsm = msgFSM(e, p_from=motor['Commissioning Date'])
fsm = msgFSM(e)
fsm.run1() # run Finite State Machine

In [ ]:
fsm.run2() # Ergebnisse verfeinern und ergänzen

In [ ]:

# Struktur zur Aufnahme mehrerer motoren
fsm_data = []
fsm_data.append({'engine':e, 'fsm':fsm})

In [ ]:
res = fsm_data[0]
res['result'] = pd.DataFrame(fsm_data[0]['fsm']._starts)
fsm.summary(res)

In [ ]:
rmodes = ['???','OFF','MANUAL','AUTO']; mw = [] ; modes = []
for mm in rmodes:
    mw.append(widgets.Checkbox(value=False, description='Mode: ' + mm))
display(widgets.VBox(mw))

In [ ]:
rsucc = [True,False]; sw = []; success=[]
for rs in rsucc:
    sw.append(widgets.Checkbox(value=False, description='Success: ' + str(rs)))
display(widgets.VBox(sw))

In [ ]:
rdf = res['result']
modes = [rmodes[i] for i,v in enumerate(mw) if v.value]
modes = rmodes if not modes else modes
success = [rsucc[i] for i,v in enumerate(sw) if v.value]
success = rsucc if not success else success

rda = rdf[:].reset_index(drop='index')
rda['count_alarms'] = rda.apply(lambda x: len(x['alarms']), axis=1)
rda['count_warnings'] = rda.apply(lambda x: len(x['warnings']), axis=1)
rda = rda[((rdf['mode'].isin(modes)) & (rdf['success'].isin(success)))].reset_index(drop='index')
print(modes, success)

In [ ]:
# special filters can be added like ... 
#rda = rda[((rda['loadramp'] < 80.0) & (rda['startpreparation'] < 300.0))].reset_index(drop='index')

#rda = rda[((rda['count_warnings'] > 0) | (rda['count_alarms'] > 0))].reset_index(drop='index')
# display the filterd & selected data
rda[fsm.filter_content + ['count_alarms', 'count_warnings']].round(2)

In [ ]:
startversuch = rda.iloc[3];
von_dt=pd.to_datetime(startversuch['starttime']); von=int(von_dt.timestamp() - fsm._pre_period)
bis_dt=pd.to_datetime(startversuch['endtime']); bis=int(bis_dt.timestamp())

cycle = 1 #sec.
data = fsm.get_period_data(von, bis, cycletime=cycle)
ftitle = f"{fsm._e} ----- Start {startversuch.name} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
print(f"von: {von_dt} = {von} bis: {bis_dt} = {bis}")
data = fsm.get_cycle_data(startversuch, max_length=None, min_length=None, cycletime=1)

In [ ]:
dset = [
    {'col':['Power_PowerAct'], 'ylim':(0,5000)},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500]}
]
dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(6,4))

In [ ]:
def local_detect_edge(data, name, kind='left'):
    fac = {'left': -1.0, 'right': 1.0}
    ldata = data[['datetime',name]]
    x0 = ldata.iloc[0]['datetime'];
    x1 = ldata.iloc[-1]['datetime'];
    edge0 = data.loc[data[name].idxmax()]
    
    try:
        if kind == 'left':
            xfac = (x1 - x0) / (edge0.datetime - x0)
        elif kind == 'right':
            xfac = (x1 - x0) / (x1 - edge0.datetime)
        else:
            raise ValueError('detect_edge: unknown kind parameter value.')
    except ZeroDivisionError:
        xfac = 0.0
    xfac = min(xfac, 5.0)
    #print(f"###### | xfac: {xfac:5.2f} | kind: {kind:>5} | name: {name}")
    lmax = ldata.loc[:,name].max() * xfac * 0.90

    data[name+'_'+kind] = data[name]+(data['datetime'] - x0)*(fac[kind] * lmax)/(x1-x0) + lmax* (1-fac[kind])/2
    
    Point = namedtuple('edge',["loc", "val"])
    edge = data.loc[data[name+'_'+kind].idxmax()]
    return  Point(edge.datetime, ldata.at[edge.name,name])

In [ ]:
dset = [
    {'col':['Various_Values_SpeedAct','Various_Values_SpeedAct_left', 'Various_Values_SpeedAct_right'], 'ylim':(0,10000)}
]
pl = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='left')
pr = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='right')
fig, ax, axes = dmyplant2.chart(data, dset, figsize=(14,8), title=ftitle)
dmyplant2.add_lines(pl.loc, [], ax, color='blue', linestyle="-.")
dmyplant2.add_lines(pr.loc, [], ax, color='blue', linestyle="-.")

In [ ]:
dset = [
    {'col':['Power_PowerAct','Power_PowerAct_left','Power_PowerAct_right'], 'ylim':(0,40000)}
]
pl = fsm.detect_edge(data, 'Power_PowerAct', kind='left')
pr = fsm.detect_edge(data, 'Power_PowerAct', kind='right')
fig, ax, axes = dmyplant2.chart(data, dset, figsize=(14,8), title=ftitle)
dmyplant2.add_lines(pl.loc, [], ax, color='blue', linestyle=":")
dmyplant2.add_lines(pr.loc, [], ax, color='purple', linestyle=":")

In [ ]:
dset = [
    {'col':['Power_PowerAct','Various_Values_SpeedAct'], '_ylim':(0,40000)},
    {'col':['Power_PowerAct_left','Power_PowerAct_right','Various_Values_SpeedAct_left', 'Various_Values_SpeedAct_right'], '_ylim':(0,40000)}
    ]

pl = fsm.detect_edge(data, 'Power_PowerAct', kind='left')
pr = fsm.detect_edge(data, 'Power_PowerAct', kind='right')
sl = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='left')
sr = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='right')
fig, ax, axes = dmyplant2.chart(data, dset, figsize=(14,8), title=ftitle)

sv_lines = [v for v in startversuch[fsm.vertical_lines_times] if v==v]
start = startversuch['starttime']; lines=list(np.cumsum(sv_lines))
dmyplant2.add_lines(start, lines, ax, color='red', linestyle="dotted")
dmyplant2.add_lines(sl.loc, [], ax, color='blue', linestyle="-.")
dmyplant2.add_lines(sr.loc, [], ax, color='purple', linestyle="-.")
dmyplant2.add_lines(pl.loc, [], ax, color='green', linestyle="-.")
dmyplant2.add_lines(pr.loc, [], ax, color='orange', linestyle="-.")

summary = pd.DataFrame(startversuch[fsm.filter_times], dtype=np.float64).round(2).T
dmyplant2.add_table(summary, ax, loc='upper left')

plt.show()

In [ ]:
dset = [
    {'col':['Power_PowerAct','Various_Values_SpeedAct'], '_ylim':(0,10000)}
]
for ii,startversuch in rda.iterrows():
        if ii < 0:
                continue
        if ii > 10:
                break
        data = fsm.get_cycle_data(startversuch, max_length=None, min_length=None, cycletime=1)

        pl = fsm.detect_edge(data, 'Power_PowerAct', kind='left')
        pr = fsm.detect_edge(data, 'Power_PowerAct', kind='right')
        sl = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='left')
        sr = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='right')

        ftitle = f"{fsm._e} ----- Start {ii} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
        #ax, ax2, idf = dmyplant2._plot(data, style='.-', figsize=(18,10), title=ftitle);
        fig, ax, axes = dmyplant2.chart(data, dset, figsize=(12,8), title=ftitle)

        ml = (data.iloc[-1]['time'] - data.iloc[0]['time']) // 1000
        sv_lines = [v for v in startversuch[fsm.vertical_lines_times]]

        start = startversuch['starttime'];
        
        nsv_lines = [v for v in sv_lines if ((v==v) and (v <= ml)) ]
        lines=list(np.cumsum(nsv_lines))
        dmyplant2.add_lines(start, lines, ax, color='red', linestyle="--")

        dmyplant2.add_lines(sl.loc, [], ax, color='green', linestyle="-")
        dmyplant2.add_lines(sr.loc, [], ax, color='green', linestyle="-.")
        dmyplant2.add_lines(pl.loc, [], ax, color='blue', linestyle="-")
        dmyplant2.add_lines(pr.loc, [], ax, color='blue', linestyle="-.")

        svdf = pd.DataFrame(sv_lines, index=fsm.vertical_lines_times, columns=['FSM']).fillna(0)
        svdf['RUN2'] = svdf['FSM']
        #if svdf.at['hochlauf','FSM'] > 0.0:
        #        svdf.at['hochlauf','RUN2'] = sl.loc.timestamp() - start.timestamp() - np.cumsum(svdf['RUN2'])['starter']
        #        svdf.at['idle','RUN2'] = svdf.at['idle','FSM'] - (svdf.at['hochlauf','RUN2'] - svdf.at['hochlauf','FSM'])
        if svdf.at['loadramp','FSM'] > 0.0:
                svdf.at['loadramp','RUN2'] = pl.loc.timestamp() - start.timestamp() - np.cumsum(svdf['RUN2'])['synchronize']
        svdf = pd.concat([
                svdf, 
                pd.DataFrame.from_dict(
                        {       'maxload':['-',pl.val],
                                'ramp':['-',(pl.val / fsm._e['Power_PowerNominal']) * 100 / svdf.at['loadramp','RUN2']],
                                'cumstarttime':[np.cumsum(svdf['FSM'])['loadramp'], np.cumsum(svdf['RUN2'])['loadramp']]
                        }, 
                        columns=['FSM','RUN2'],
                        orient='index')]
                )


        #summary = pd.DataFrame(startversuch[fsm.filter_times], dtype=np.float64).round(2).T
        summary = pd.DataFrame(svdf['FSM']).round(1).T
        dmyplant2.add_table(summary, ax, loc='upper left')
        #display(HTML(summary.to_html(escape=False)))
        display(HTML(svdf.round(2).T.to_html(escape=False)))


        for i, al in enumerate(startversuch['alarms']):
                print(f"{al['state']:16} {fsm.msgtxt(al['msg'],i)}")

        for i, w in enumerate(startversuch['warnings']):
                print(f"{w['state']:16} {fsm.msgtxt(w['msg'],i)}")
        
        #fsm.plot_cycle(startversuch, max_length=600, ylim=(0,2500), cycletime=1, style='.-', figsize=(10,6), 
        #    title=f"{fsm._e} ----- Start {startversuch.name} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}");
        plt.show();

In [ ]:
rda.describe().round(1).T

In [ ]:
svdf

In [ ]:
longer_than_300s = rda[fsm.filter_period + fsm.filter_content][rda['cumstarttime'] > 300.0]
longer_than_300s[fsm.filter_content].round(2)

In [ ]:
longer_than_300s.describe().round(2).T

In [ ]:
load_ramp_less_than_100s = rda[fsm.filter_period + fsm.filter_content][rda['loadramp'] < 100.0]
lfsm = load_ramp_less_than_100s[fsm.filter_content].reset_index(drop='index').round(2)
lfsm

In [ ]:
rda.reset_index(drop='index').iloc[10];

In [ ]:
for i, r in load_ramp_less_than_100s.reset_index(drop='index').iterrows():
    fsm.plot_cycle(r, max_length=8*60, ylim=(0,2500), cycletime=1, marker=None,figsize=(20,12), title=f"{i:3d} - {fsm._e} {r['starttime'].round('S')}")
    plt.show()


In [ ]:
load_ramp_less_than_100s.describe().round(2).T

In [ ]:
rda[['startpreparation','starter','hochlauf','idle','synchronize','cumstarttime','loadramp','targetoperation']].hist(figsize = (20,12), bins=80, layout=(3,3));
#rda.hist(figsize = (20,12), bins=100, layout=(4,2));

In [ ]:
nalarms = []
ct = 0
ct2 = 0
mini = 0
maxi = 1
for i,c in rdf.iterrows():
    if len(c['alarms']) > 0 and not c['success']:
        ct += 1
        print(f"\nStartversuch: {i}, Success: {c['success']}")
        for a in c['alarms']:
            nalarms.append(a['msg'])
            _txt = f"{ct2} {c['mode']:15} {a['state']:20} {a['msg']['timestamp']} {pd.to_datetime(int(a['msg']['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {a['msg']['name']} {a['msg']['message']}"
            print(_txt)
            if ct2 >= mini and ct2 <= maxi:
                fsm.plot_cycle(c, ylim=(0,2500), cycletime=1, marker=None,figsize=(20,12), title=f"{i:3d} - {fsm._e} {_txt}")
                plt.show()
            ct2 += 1

print(f"""
***********************************
** {ct:3} nicht erfolgreiche Starts **
***********************************
""")

In [ ]:
al = pd.DataFrame(fsm._pareto(nalarms))
fig = plt.figure();
color = 'purple'

if not al.empty:
    al['msg'] = al['msg'] + ' (' + al['name'] + ')'
    al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title='Alarms in not successful Starts');
    plt.plot();

In [ ]:
phases = [['startpreparation','starter','hochlauf','idle','synchronize','loadramp','targetoperation'],'startpreparation','starter','hochlauf','idle','synchronize','loadramp','targetoperation']
fig = plt.figure();
color = 'red'

for phase in phases:
    al = fsm.alarms_pareto(phase)[:30]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
fig = plt.figure();
color = 'orange'

for phase in phases:
    al = fsm.warnings_pareto(phase)[:30]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
mfn = e._fname + '_messages.txt'
fsm.save_messages(mfn)
print(mfn)

In [ ]:
any(fsm._messages['name'] == '9047')

In [ ]:
#fsm.store()